In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_horizVert_wavelet_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8
2,2155.33,-0.01,0.03,0.00,0.00,-0.00,0.00
3,-0.01,676.37,-0.01,0.01,0.00,0.00,-0.00
4,0.03,-0.01,188.36,0.00,-0.00,0.00,-0.00
5,0.00,0.01,0.00,43.77,-0.00,-0.00,0.00
6,0.00,0.00,-0.00,-0.00,8.85,0.00,0.00
7,-0.00,0.00,0.00,-0.00,0.00,1.68,-0.00
8,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.27


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8
2,1.00000,-0.00001,0.00005,0.00000,0.00001,-0.00006,0.00001
3,-0.00001,1.00000,-0.00001,0.00003,0.00000,0.00008,-0.00005
4,0.00005,-0.00001,1.00000,0.00003,-0.00001,0.00003,-0.00001
5,0.00000,0.00003,0.00003,1.00000,-0.00001,-0.00001,0.00003
6,0.00001,0.00000,-0.00001,-0.00001,1.00000,0.00002,0.00006
7,-0.00006,0.00008,0.00003,-0.00001,0.00002,1.00000,-0.00001
8,0.00001,-0.00005,-0.00001,0.00003,0.00006,-0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.04757910906369772

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[6.65781019e+05 6.35777499e+04 4.73142487e+03 2.39737177e+02
 8.74399043e+00 2.39896275e-01 4.60317855e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.998229,-0.057637,-0.014323,-0.003304,-0.000667,-0.000129,-0.000020
1,0.056614,0.996445,-0.060979,-0.012906,-0.002598,-0.000490,-0.000081
2,0.017493,0.059130,0.996099,-0.061984,-0.011726,-0.002220,-0.000361
3,0.005037,0.016091,0.060040,0.995467,-0.070638,-0.012721,-0.002057
4,0.001345,0.004289,0.015422,0.068139,0.992577,-0.098295,-0.015095
5,0.000378,0.001196,0.004307,0.018565,0.093444,0.982123,-0.162290
6,0.000125,0.000398,0.001429,0.006148,0.030405,0.160018,0.986625


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00177082, 0.00355491, 0.00390104, 0.00453273, 0.00742292,
       0.01787708, 0.01337452])